# Youtube

Mục tiêu : cần tìm hiểu tất cả các video trên youtube về Knowledge Graph từ các hội nghị ICML
ICML : International Conference Machine Learning (Hội nghị quốc tế về Máy học)

### SimpleE

* SimplE Embedding for Link Prediction in Knowledge Graphs
https://www.youtube.com/watch?v=tdOHzJgw_CE

Embeddings are learned using existing links 

* KGCN | Knowledge Graph Convolutional Networks - Machine Learning over a Knowledge Graph 
https://www.youtube.com/watch?v=JlcGfwb6CDE

Bài viết cùng chủ đề 
https://blog.grakn.ai/kgcns-machine-learning-over-knowledge-graphs-with-tensorflow-a1d3328b8f02

In [1]:
from py2neo import Graph

graph = Graph()

ModuleNotFoundError: No module named 'py2neo'

In [2]:
!pip install py2neo 

  Created wheel for py2neo: filename=py2neo-4.3.0-py2.py3-none-any.whl size=88311 sha256=2afbb86740a3db83d1618ae4536bf142cb0abf9e8f04496134c7fb3bc36c24f5
  Stored in directory: c:\users\thanh\appdata\local\pip\cache\wheels\64\cb\88\3da99c715827eeffd83bddaa657023c118aa9d3282dbc697e9
  Created wheel for neobolt: filename=neobolt-1.7.17-cp37-cp37m-win_amd64.whl size=150504 sha256=21b06b0381b6ed836052662793c81d88bb3967338a9ce506269f09d3cb78a891
  Stored in directory: c:\users\thanh\appdata\local\pip\cache\wheels\9d\46\80\bbc5e90b72fcad84080d47919dddb8c90437de148a8501b434
  Created wheel for neotime: filename=neotime-1.7.4-py3-none-any.whl size=20546 sha256=aba7b23809f1d6a28d41c7d2d1794860e29cc346eaf0973909f39ae019f26578
  Stored in directory: c:\users\thanh\appdata\local\pip\cache\wheels\1b\53\64\de72baae858a94a3ed2c6b1117edf988b2b8d57b798097e5b3
Successfully built py2neo neobolt neotime
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.3
    Uninst

ERROR: spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [4]:
import matplotlib.pyplot as plt

In [10]:
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456"))

In [11]:
graph.delete_all()

In [12]:
from py2neo import Node

nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola")
pepsi = Node("Manufacturer", name="Pepsi")

graph.create(nicole | drew | mtdew | cokezero | coke | pepsi)

ClientError: SyntaxError: The old parameter syntax `{param}` is no longer supported. Please use `$param` instead (line 5, column 11 (offset: 74))
"    LIMIT {limit}"
           ^

In [23]:
from IPython.display import IFrame
import json
import uuid

def vis_network(nodes, edges, physics=False):
    html = """
    <html>
    <head>
      <script type="text/javascript" src="../lib/vis/dist/vis.js"></script>
      <link href="../lib/vis/dist/vis.css" rel="stylesheet" type="text/css">
    </head>
    <body>
    <div id="{id}"></div>
    <script type="text/javascript">
      var nodes = {nodes};
      var edges = {edges};
      var container = document.getElementById("{id}");
      var data = {{
        nodes: nodes,
        edges: edges
      }};
      var options = {{
          nodes: {{
              shape: 'dot',
              size: 25,
              font: {{
                  size: 14
              }}
          }},
          edges: {{
              font: {{
                  size: 14,
                  align: 'middle'
              }},
              color: 'gray',
              arrows: {{
                  to: {{enabled: true, scaleFactor: 0.5}}
              }},
              smooth: {{enabled: false}}
          }},
          physics: {{
              enabled: {physics}
          }}
      }};
      var network = new vis.Network(container, data, options);
    </script>
    </body>
    </html>
    """

    unique_id = str(uuid.uuid4())
    html = html.format(id=unique_id, nodes=json.dumps(nodes), edges=json.dumps(edges), physics=json.dumps(physics))

    filename = "figure/graph-{}.html".format(unique_id)

    file = open(filename, "w")
    file.write(html)
    file.close()

    return IFrame(filename, width="100%", height="400")

def draw(graph, options, physics=False, limit=100):
    # The options argument should be a dictionary of node labels and property keys; it determines which property
    # is displayed for the node label. For example, in the movie graph, options = {"Movie": "title", "Person": "name"}.
    # Omitting a node label from the options dict will leave the node unlabeled in the visualization.
    # Setting physics = True makes the nodes bounce around when you touch them!
    query = """
    MATCH (n)
    WITH n, rand() AS random
    ORDER BY random
    LIMIT {limit}
    OPTIONAL MATCH (n)-[r]->(m)
    RETURN n AS source_node,
           id(n) AS source_id,
           r,
           m AS target_node,
           id(m) AS target_id
    """

    data = graph.run(query, limit=limit)

    nodes = []
    edges = []

    def get_vis_info(node, id):
        node_label = list(node.labels())[0]
        prop_key = options.get(node_label)
        vis_label = node.properties.get(prop_key, "")

        return {"id": id, "label": vis_label, "group": node_label, "title": repr(node.properties)}

    for row in data:
        source_node = row[0]
        source_id = row[1]
        rel = row[2]
        target_node = row[3]
        target_id = row[4]

        source_info = get_vis_info(source_node, source_id)

        if source_info not in nodes:
            nodes.append(source_info)

        if rel is not None:
            target_info = get_vis_info(target_node, target_id)

            if target_info not in nodes:
                nodes.append(target_info)

            edges.append({"from": source_info["id"], "to": target_info["id"], "label": rel.type()})

    return vis_network(nodes, edges, physics=physics)

In [26]:
!pip install py2neo==5.0b1

  Created wheel for py2neo: filename=py2neo-5.0b1-py2.py3-none-any.whl size=111870 sha256=e60a8f528ce530567a28e3f3cc7bad2114f1f205fed2a799054b575db474a7d9
  Stored in directory: c:\users\thanh\appdata\local\pip\cache\wheels\57\c8\6d\32f5b79e1a95aa2398739e6673a7dde64cdd77fe31dae8f3cc
Successfully built py2neo
  Attempting uninstall: py2neo
    Found existing installation: py2neo 4.3.0
    Uninstalling py2neo-4.3.0:
      Successfully uninstalled py2neo-4.3.0


In [28]:
options = {"Person": "name", "Drink": "name", "Manufacturer": "name"}
draw(graph, options)

ClientError: SyntaxError: The old parameter syntax `{param}` is no longer supported. Please use `$param` instead (line 5, column 11 (offset: 74))
"    LIMIT {limit}"
           ^